In [ ]:
import os
if(os.path.exists("./dqn_example.py")):
    os.chdir("..")
    
import matplotlib.pyplot as plt
import pandas as pd
from absl import flags, app

from run import eval

In [ ]:
root = './runs/'
folder = 'learning_rate/'

eval_results = dict()
def main(argv):
    if(not os.path.exists(os.path.join(root, folder, 'eval.csv'))):
        eval_dirs = os.listdir(f'{root}{folder}')
        for eval_dir in eval_dirs:
            eval_dir = os.path.join(folder, eval_dir, 'checkpoint-last.pt')
            eval_results[eval_dir] = eval(eval_dir)
            df = pd.DataFrame.from_dict(eval_results)
            df.to_csv(os.path.join(root, folder, 'eval.csv'))
            quit()
        else:
            quit()

app.run(main, argv = None)

In [ ]:
def show_boxplot(folder : str, title : str, ylabel :str, labels : list = None):
    df = pd.read_csv(os.path.join(root, folder, 'eval.csv'), index_col=0)
    fig, axs = plt.subplots(figsize=(22, 5))
    axs.set_title(title)
    if(labels == None):
        labels = df.columns
    # if(isinstance(labels[0], float)):
    #     df.columns = labels
    #     df = df.sort_index()
    axs.boxplot(df, labels=labels, vert=False)
    axs.set_xlabel("reward")
    axs.set_ylabel(ylabel)
    plt.show()

def get_labels(folder, key):
    labels = dict()
    # extract learning_rates from flags.txt
    date_dirs = os.listdir(os.path.join(root, folder))
    for date_dir in date_dirs:
        flags_dir = os.path.join(root, folder, date_dir, 'flags.txt')
        if(os.path.exists(flags_dir)):
            with open(flags_dir, 'r') as f:
                lines = f.readlines()
                for line in lines:
                    if(line.startswith(f'--{key}')):
                        labels[date_dir] = line.split('=')[1].strip()
    return labels
    

In [ ]:
show_boxplot('warmup', 'Evaluation Results [n warmup steps]', "n warmup steps", get_labels('warmup', 'warmup_steps').values()) # ['0', '10k', '50k', '300k']

In [ ]:
show_boxplot('learning_rate', 'Evaluation Results [learning rate]', "learning rate", [float(x) for x in get_labels('learning_rate', 'learning_rate').values()])